In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from selenium import webdriver
import main
import visuals
import seaborn as sns
import chromedriver_autoinstaller
from datetime import datetime
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
from matplotlib.colors import to_rgba
import matplotlib as mpl
from collections import defaultdict

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 30)

In [ ]:
chromedriver_autoinstaller.install()

- You'll only need to change the link of the game and the name of the player you're interested in

## Pull Data 

In [ ]:
chromedriver_autoinstaller.install()

if __name__ == "__main__":
    driver = webdriver.Chrome()
    
# whoscored match centre url of the required match (Example: Barcelona vs Sevilla)
url = "https://www.whoscored.com/Matches/1640867/Live/England-Premier-League-2022-2023-Chelsea-Crystal-Palace"
match_data = main.getMatchData(driver, url, close_window=True)

# Match dataframe containing info about the match
matches_df = main.createMatchesDF(match_data)

# Events dataframe      
events_df = main.createEventsDF(match_data)

# match Id
matchId = match_data['matchId']

# Information about respective teams as dictionary
home_data = matches_df['home'][matchId]
away_data = matches_df['away'][matchId]
score = matches_df['score'][matchId]
score = ' ' + score.replace(':', '-') + ' '

home_name = home_data['name']
away_name = away_data['name']
date = match_data['startDate'].split('T')[0]
date_ = datetime.strptime(date, '%Y-%m-%d').date()
date_str = date_.strftime('%d/%m/%Y')

## Data Parsing

In [ ]:
events_df["passRecipient"] = events_df['playerName'].shift(-1)
events_df['teamRecipient'] = events_df['teamId'].shift(-1)
column_to_move = events_df.pop("passRecipient")
events_df.insert(26, "passRecipient", column_to_move)
column_to_move = events_df.pop("teamRecipient")
events_df.insert(27, "teamRecipient", column_to_move)

## Passing 
- Player in question is Conor Gallagher

In [ ]:
df_passes = events_df[(events_df['playerName'] == 'Conor Gallagher') & (events_df['type'] == 'Pass')].reset_index(drop=True)
df_passes2 = df_passes[['playerName', 'x', 'y', 'endX', 'endY', 'outcomeType']]

df_passes2['isKeyPass'] = ''
keyPassCols = ['keyPassLong', 'keyPassShort', 'keyPassCross','keyPassCorner', 'keyPassThroughball', 'keyPassFreekick', 'keyPassThrowin', 'keyPassOther']
df_passes2['isProg'] = ''
df_passes2['isPassIntoBox'] = ''

for i in range(len(df_passes)):
    for col in keyPassCols:
        if df_passes[col][i] == True:
            df_passes2['isKeyPass'][i] = 1
            break
        else:
            df_passes2['isKeyPass'][i] = 0
                
    x_0 = 120*(df_passes2['x'][i])/100
    y_0 = 80*(df_passes2['y'][i])/100
    x_1 = 120*(df_passes2['endX'][i])/100
    y_1 = 80*(df_passes2['endY'][i])/100
    
    beg_dist = np.sqrt((120 - x_0)**2 + (40 - y_0)**2)
    end_dist = np.sqrt((120 - x_1)**2 + (40 - y_1)**2)

    if end_dist/beg_dist <= 0.75:
        df_passes2['isProg'][i] = 1
    else:
        df_passes2['isProg'][i] = 0
            
    if (x_1 >= 102) and (18 <= y_1 <= 62):
        df_passes2['isPassIntoBox'][i] = 1
    else:
        df_passes2['isPassIntoBox'][i] = 0
        
df_passes2
    

In [ ]:
df_passes_succ = df_passes2[(df_passes2['outcomeType'] == 'Successful') & (df_passes2['isKeyPass'] == 0) & 
                            (df_passes2['isProg'] == 0) & (df_passes2['isPassIntoBox'] == 0)].reset_index(drop=True)
df_passes_kp = df_passes2[(df_passes2['outcomeType'] == 'Successful') & (df_passes2['isKeyPass'] == 1)].reset_index(drop=True)
df_passes_bp = df_passes2[(df_passes2['outcomeType'] == 'Successful') & (df_passes2['isKeyPass'] == 0) & 
                          (df_passes2['isPassIntoBox'] == 1)].reset_index(drop=True)
df_passes_pp = df_passes2[(df_passes2['outcomeType'] == 'Successful') & (df_passes2['isKeyPass'] == 0) & 
                          (df_passes2['isPassIntoBox'] == 0) & (df_passes2['isProg'] == 1)].reset_index(drop=True)

In [ ]:
passes_attempted = len(df_passes2)
passes_succ_rate = round(len(df_passes2[df_passes2['outcomeType'] == 'Successful'])/passes_attempted*100)
kp_attempted = df_passes2['isKeyPass'].sum()
kp_succ = df_passes2[df_passes2['outcomeType'] == 'Successful']['isKeyPass'].sum()
bp_attempted = df_passes2['isPassIntoBox'].sum()
bp_succ = df_passes2[df_passes2['outcomeType'] == 'Successful']['isPassIntoBox'].sum()
pp_attempted = df_passes2['isProg'].sum()
pp_succ = df_passes2[df_passes2['outcomeType'] == 'Successful']['isProg'].sum()
pp_succ

In [ ]:
playerName = str(df_passes2['playerName'].unique()[0])
playerName

## Carry Data 
- Opta match data doesn't contain carry data
- So need to derive it ourselves (not gonna be 100% accurate)

In [ ]:
df_carries = pd.DataFrame()
min_len = 3
max_len = 60
min_time = 1
max_time = 12

for idx, match_event in events_df.iterrows():
    if idx == len(events_df) - 1:
        break
    
    cur_team = match_event['teamId']
    idx_next = idx + 1
    events_df_next = events_df.loc[idx_next]
    take_ons = 0
    loop = True
    count = 0 
    
    while loop:
        next_events = events_df.loc[idx_next]
        
        if next_events['type'] == 'TakeOn' and next_events['outcomeType'] == 'Successful':
            take_ons += 1
            loop = True
        
        elif (next_events['type'] == 'TakeOn' and next_events['outcomeType'] == 'Unsuccessful') or (next_events['teamId'] != cur_team and next_events['type'] == 'Challenge' and next_events['outcomeType'] == 'Unsuccessful') or (next_events['type'] == 'Foul'):
            loop = True
            
        else:
            loop = False
            
        idx_next += 1
        count += 0 
        

    same_team = cur_team == next_events['teamId']
    not_ball_touch = match_event['type'] != 'BallTouch'
    dx = 105 * (match_event['endX'] - next_events['x'])/100
    dy = 68 * (match_event['endY'] - next_events['y'])/100
    min_dist = dx**2 + dy**2 >= min_len**2
    max_dist = dx**2 + dy**2 <= max_len**2
    same_period = match_event['period'] == next_events['period']
    
    valid_carry = same_team & not_ball_touch & min_dist & max_dist & same_period 
    
    if valid_carry:
        carry = pd.DataFrame()
        prev = match_event
        nxt = next_events
        carry.loc[0, 'eventId'] = prev['eventId'] + count
        carry['id'] = prev['id'] + count
        carry['teamId'] = nxt['teamId']
        carry['x'] = prev['endX']
        carry['y'] = prev['endY']
        carry['period'] = nxt['period']
        carry['type'] = carry.apply(lambda x: {'value': 99, 'displayName': 'Carry'}, axis=1)
        carry['outcomeType'] = 'Successful'
        carry['qualifiers'] = carry.apply(
            lambda x: {'type': {'value': 999, 'displayName': 'takeOns'}, 'value': str(take_ons)}, axis=1)
        carry['satisfiedEventsTypes'] = carry.apply(lambda x: [], axis=1)
        carry['isTouch'] = True
        carry['playerId'] = nxt['playerId']
        carry['playerName'] = nxt['playerName']
        carry['endX'] = nxt['x']
        carry['endY'] = nxt['y']
        
        df_carries = pd.concat([df_carries, carry], ignore_index=True, sort=False)


In [ ]:
df_carry = df_carries[df_carries['playerName'] == playerName].reset_index(drop=True)
df_carry['isProg'] = ''
df_carry['isCarryIntoBox'] = ''

for i in range(len(df_carry)):
    x_0 = 120*df_carry['x'][i]/100
    y_0 = 80*df_carry['y'][i]/100
    x_1 = 120*df_carry['endX'][i]/100
    y_1 = 80*df_carry['endY'][i]/100
    
    beg_dist = np.sqrt((120 - x_0)**2 + (40 - y_0)**2)
    end_dist = np.sqrt((120 - x_1)**2 + (40 - y_1)**2)

    if end_dist/beg_dist <= 0.75:
        df_carry['isProg'][i] = 1
    else:
        df_carry['isProg'][i] = 0
        
    if (x_1 >= 102) and (18 <= y_1 <= 62):
        df_carry['isCarryIntoBox'][i] = 1
    else:
        df_carry['isCarryIntoBox'][i] = 0
            

In [ ]:
df_succ_carries = df_carry[(df_carry['isProg'] == 0) & df_carry['isCarryIntoBox'] == 0]
carries = len(df_carry)
df_carry_box = df_carry[df_carry['isCarryIntoBox'] == 1]
bc = df_carry['isCarryIntoBox'].sum()
df_carry_prog = df_carry[(df_carry['isProg'] == 1) & (df_carry['isCarryIntoBox'] == 0)]
pc = df_carry['isProg'].sum()
take_ons_succ = events_df[(events_df['playerName'] == playerName) & (events_df['type'] == 'TakeOn') & 
                          (events_df['outcomeType'] == 'Successful')]
take_ons_attempted = len(events_df[(events_df['playerName'] == playerName) & (events_df['type'] == 'TakeOn')])

## Defensive Actions 

In [ ]:
df_player = events_df[events_df['playerName'] == playerName].reset_index(drop=True)

df_tackles = df_player[df_player['type'] == 'Tackle'].reset_index(drop=True)
df_succ_tackles = df_tackles[df_tackles['outcomeType'] == 'Successful']
tackles_succ = len(df_succ_tackles)
tackles_attempted = len(df_tackles)

df_aerial = df_player[df_player['type'] == 'Aerial'].reset_index(drop=True)
df_succ_aerial = df_aerial[df_aerial['outcomeType'] == 'Successful']
aerials_succ = len(df_succ_aerial)
aerials_attempted = len(df_aerial)

df_interception = df_player[df_player['type'] == 'Interception'].reset_index(drop=True)
interceptions = len(df_interception)

df_clearance = df_player[df_player['type'] == 'Clearance'].reset_index(drop=True)
clearances = len(df_clearance)

df_foul = df_player[(df_player['type'] == 'Foul') & (df_player['foulCommitted'] == True)].reset_index(drop=True)
fouls = len(df_foul)

df_recovery = df_player[df_player['type'] == 'Recovery'].reset_index(drop=True)
recoveries = len(df_recovery)

df_blockedpass = df_player[df_player['type'] == 'BlockedPass'].reset_index(drop=True)
blockedpasses = len(df_blockedpass)

df_blockedshot = events_df[(events_df['type'] == 'SavedShot') 
                           & (events_df['passRecipient'] == playerName)].reset_index(drop=True)
blockedshots = len(df_blockedshot)

## Plot

In [ ]:
pitch = VerticalPitch(pitch_type='opta', pitch_color='#2B2B2B', line_color='#c7d5cc',pad_top=0, pad_left = 7.5,
                      pad_bottom = 40, pad_right = 7.5, pitch_width=80, pitch_length=120,
                      linewidth=0.6)
fig, axs = pitch.grid(figheight=15, ncols=3, axis=False, endnote_height=0.02)
fig.set_facecolor("#2B2B2B")

axs['title'].text(0, 0.7, playerName.upper() + ' Performance '.upper(), 
                  ha='left', va='center', color='white',
                  fontsize=30, fontname='Sans Serif')
axs['title'].text(0, 0.50, 
                  home_name.upper() + ' vs ' + away_name.upper() + '  |  ' + date_str + '  |  ' + ' Created by'.upper() + ' ...', 
                  ha='left', va='center', color='white',
                  fontsize=20, fontname='Sans Serif')

# Pass Plot
pass_arrow_kp = pitch.lines(df_passes_kp.x, df_passes_kp.y,
                          df_passes_kp.endX, df_passes_kp.endY, lw=5,
                          color='lawngreen', zorder=8,comet=True, ax=axs['pitch'][0])

pass_nodes_kp = pitch.scatter(df_passes_kp.endX, df_passes_kp.endY, s=100,
                           color='lawngreen', linewidth=1, zorder=7, ax=axs['pitch'][0])

pass_arrow_bp = pitch.lines(df_passes_bp.x, df_passes_bp.y,
                          df_passes_bp.endX, df_passes_bp.endY, lw=5,
                          color='gold', zorder=6,comet=True, ax=axs['pitch'][0])

pass_nodes_bp = pitch.scatter(df_passes_bp.endX, df_passes_bp.endY, s=100,
                           color='gold', linewidth=1, zorder=5, ax=axs['pitch'][0])

pass_arrow_pp = pitch.lines(df_passes_pp.x, df_passes_pp.y,
                          df_passes_pp.endX, df_passes_pp.endY, lw=5,
                          color='#00B2EE', zorder=4,comet=True, ax=axs['pitch'][0])

pass_nodes_pp = pitch.scatter(df_passes_pp.endX, df_passes_pp.endY, s=100,
                           color='#00B2EE', linewidth=1, zorder=3, ax=axs['pitch'][0])

pass_arrow_sp = pitch.lines(df_passes_succ.x, df_passes_succ.y,
                          df_passes_succ.endX, df_passes_succ.endY, lw=5,
                          edgecolor='#575757',color='#2B2B2B', zorder=1,comet=True, ax=axs['pitch'][0])

pass_nodes_sp = pitch.scatter(df_passes_succ.endX, df_passes_succ.endY, s=100,
                           edgecolor='#575757', linewidth=2, color='#2B2B2B', zorder=2, ax=axs['pitch'][0])

pitch.annotate(text='Passes: '.upper() + str(passes_attempted) + ' (' + str(passes_succ_rate) + '% success rate)'.upper() , 
                      xytext=(-4, 100), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='white', ax=axs['pitch'][0])
pitch.annotate(text='Key Passes: '.upper() + str(kp_succ) + ' (' + str(kp_attempted) + ' attempted)'.upper() , 
                      xytext=(-8, 100), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='lawngreen', ax=axs['pitch'][0])
pitch.annotate(text='Passes Into Box: '.upper() + str(bp_succ) + ' (' + str(bp_attempted) + ' attempted)'.upper() , 
                      xytext=(-12, 100), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='gold', ax=axs['pitch'][0])
pitch.annotate(text='Progressive Passes: '.upper() + str(pp_succ) + ' (' + str(pp_attempted) + ' attempted)'.upper() , 
                      xytext=(-16, 100), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='#00B2EE', ax=axs['pitch'][0])

pitch.annotate(text='Pass Plot'.upper(), xytext=(104,100), xy=(60,40), color='White',
                  va='center', ha='left', fontsize=20, fontname='Sans Serif', ax=axs['pitch'][0])

# Carry Plot

pass_arrow_sp = pitch.lines(df_succ_carries.x, df_succ_carries.y,
                            df_succ_carries.endX, df_succ_carries.endY, lw=5,
                            edgecolor='#575757', linestyle = '--', color='#2B2B2B', zorder=1, 
                            ax=axs['pitch'][1])
 
pass_nodes_sp = pitch.scatter(df_succ_carries.endX, df_succ_carries.endY, s=100,
                              edgecolor='#575757', linewidth=2, color='#2B2B2B', zorder=2, ax=axs['pitch'][1])

pitch.annotate(text='Carries: '.upper() + str(carries), 
                      xytext=(-4, 100), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='white', ax=axs['pitch'][1])

pass_arrow_bc = pitch.lines(df_carry_box.x, df_carry_box.y,
                          df_carry_box.endX, df_carry_box.endY, lw=5,
                          color='gold', zorder=6,linestyle = '--', ax=axs['pitch'][1])

pass_nodes_bc = pitch.scatter(df_carry_box.endX, df_carry_box.endY, s=100,
                           color='gold', linewidth=1, zorder=5, ax=axs['pitch'][1])

pitch.annotate(text='Carries Into Box: '.upper() + str(bc) , 
                      xytext=(-12, 100), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='gold', ax=axs['pitch'][1])

pass_arrow_pc = pitch.lines(df_carry_prog.x, df_carry_prog.y,
                          df_carry_prog.endX, df_carry_prog.endY, lw=5,
                          color='#00B2EE', zorder=4,linestyle='--', ax=axs['pitch'][1])

pass_nodes_pc = pitch.scatter(df_carry_prog.endX, df_carry_prog.endY, s=100,
                           color='#00B2EE', linewidth=1, zorder=3, ax=axs['pitch'][1])

pitch.annotate(text='Progressive Carries: '.upper() + str(pc) , 
                      xytext=(-16, 100), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='#00B2EE', ax=axs['pitch'][1])


pass_nodes_to = pitch.scatter(take_ons_succ.x, take_ons_succ.y, s=400,
                           color='lawngreen', linewidth=1, marker='*', zorder=7, ax=axs['pitch'][1])

pitch.annotate(text= u"\u2605", 
                      xytext=(-8, 100), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=24, color='lawngreen', ax=axs['pitch'][1])

pitch.annotate(text=  ' Take Ons: '.upper() + str(len(take_ons_succ)) + ' (' + str(take_ons_attempted) + ' Attempted)', 
                      xytext=(-8, 94), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='lawngreen', ax=axs['pitch'][1])

pitch.annotate(text='Carry Plot'.upper(), xytext=(104,100), xy=(60,40), color='White',
                  va='center', ha='left', fontsize=20, fontname='Sans Serif', ax=axs['pitch'][1])



tackles = pitch.scatter(df_succ_tackles.x, df_succ_tackles.y, s=400,
                           color='skyblue', linewidth=1, marker='*', zorder=7, ax=axs['pitch'][2])
marker_fouls = pitch.scatter(-3.75, 97, s=200,
                           color='skyblue', linewidth=1, marker='*', zorder=7, ax=axs['pitch'][2])
pitch.annotate(text=' Tackles: '.upper() + str(tackles_succ) + ' (' + str(tackles_attempted) + ' Attempted)', 
                      xytext=(-4, 95), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='skyblue', ax=axs['pitch'][2])


fouls = pitch.scatter(df_foul.x, df_foul.y, s=400,
                           color='crimson', linewidth=1, marker='X', zorder=7, ax=axs['pitch'][2])
marker_fouls = pitch.scatter(-3.75, 37, s=200,
                           color='crimson', linewidth=1, marker='X', zorder=7, ax=axs['pitch'][2])
pitch.annotate(text=' Fouls: '.upper() + str(len(df_foul)) , 
                      xytext=(-4, 34.5), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='crimson', ax=axs['pitch'][2])


interceptions = pitch.scatter(df_interception.x, df_interception.y, s=400,
                           color='gold', linewidth=1, marker='^', zorder=7, ax=axs['pitch'][2])
marker_interceptions = pitch.scatter(-7.75, 97, s=200,
                           color='gold', linewidth=1, marker='^', zorder=7, ax=axs['pitch'][2])
pitch.annotate(text=' Interceptions: '.upper() + str(len(df_interception)) , 
                      xytext=(-8, 95), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='gold', ax=axs['pitch'][2])


recoveries = pitch.scatter(df_recovery.x, df_recovery.y, s=400,
                           color='limegreen', linewidth=1, marker='o', zorder=7, ax=axs['pitch'][2])
marker_recovery = pitch.scatter(-7.75, 37, s=200,
                           color='limegreen', linewidth=1, marker='o', zorder=7, ax=axs['pitch'][2])
pitch.annotate(text=' Recoveries: '.upper() + str(len(df_recovery)) , 
                      xytext=(-8, 34.5), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='limegreen', ax=axs['pitch'][2])


aerials = pitch.scatter(df_succ_aerial.x, df_succ_aerial.y, s=400,
                           color='azure', linewidth=1, marker='p', zorder=7, ax=axs['pitch'][2])
marker_aerials = pitch.scatter(-11.75, 97, s=200,
                           color='azure', linewidth=1, marker='p', zorder=7, ax=axs['pitch'][2])
pitch.annotate(text=' Aerials: '.upper() + str(len(df_succ_aerial)) + ' (' + str(len(df_aerial)) + ' Attempted)' , 
                      xytext=(-12, 95), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='azure', ax=axs['pitch'][2])


clearances = pitch.scatter(df_clearance.x, df_clearance.y, s=400,
                           color='mediumorchid', linewidth=1, marker='D', zorder=7, ax=axs['pitch'][2])
marker_clearance = pitch.scatter(-11.75, 37, s=150,
                           color='mediumorchid', linewidth=1, marker='D', zorder=7, ax=axs['pitch'][2])
pitch.annotate(text=' Clearances: '.upper() + str(len(df_clearance)) , 
                      xytext=(-12, 34.5), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='mediumorchid', ax=axs['pitch'][2])


blockedpass = pitch.scatter(df_blockedpass.x, df_blockedpass.y, s=400,
                           color='#FF6103', linewidth=1, marker='s', zorder=7, ax=axs['pitch'][2])
marker_blockedpass = pitch.scatter(-15.75, 97, s=200,
                           color='#FF6103', linewidth=1, marker='s', zorder=7, ax=axs['pitch'][2])
pitch.annotate(text=' Blocked Pass: '.upper() + str(len(df_blockedpass)) , 
                      xytext=(-16, 95), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='#FF6103', ax=axs['pitch'][2])


blockedshot = pitch.scatter(df_blockedshot.x, df_blockedshot.y, s=400,
                           color='#FF6103', linewidth=1, marker='H', zorder=7, ax=axs['pitch'][2])
marker_blockedshot = pitch.scatter(-15.75, 37, s=200,
                           color='#FF6103', linewidth=1, marker='H', zorder=7, ax=axs['pitch'][2])
pitch.annotate(text=' Blocked Shot: '.upper() + str(len(df_blockedshot)) , 
                      xytext=(-16, 34.5), xy=(60, 40), ha='left', va='center',
                      fontname = 'Sans Serif', fontsize=15, color='#FF6103', ax=axs['pitch'][2])

pitch.annotate(text='Defensive Action Plot'.upper(), xytext=(104,100), xy=(60,40), color='White',
                  va='center', ha='left', fontsize=20, fontname='Sans Serif', ax=axs['pitch'][2])

fig.savefig('action_plot.png', dpi=300)